In [1]:
using BenchmarkTools
using Distances
using MLDatasets
using NNDescent
using NNDescent: brute_knn, brute_search, NNTuple

In [2]:
recall(nn, true_nn) = sum(_recall(nn[:,i], true_nn[:,i]) for i in 1:size(nn,2))/size(nn,2)
_recall(π, πₜ) = length(intersect(π, πₜ))/length(πₜ)

_recall (generic function with 1 method)

In [3]:
train_x, train_y = FashionMNIST.traindata()
test_x, test_y = FashionMNIST.testdata()
train_x = reshape(train_x, size(train_x)[1]*size(train_x)[2], size(train_x)[3])
test_x = reshape(test_x, size(test_x)[1]*size(test_x)[2], size(test_x)[3])

data = [convert.(Float32, train_x[:,i]) for i = 1:size(train_x, 2)]
data = data[1:5000]
queries = [convert.(Float32, test_x[:,i]) for i = 1:size(test_x, 2)]
queries = queries[1:500];

In [4]:
knn_graph = DescentGraph(data, 10)
nn = getindex.(knn_graph.graph, 1);

In [5]:
brute_graph = brute_knn(data, Euclidean(), 10)
true_nn = getindex.(brute_graph, 1);

In [6]:
recall(nn, true_nn)

0.99744

In [7]:
true_idx, true_dist = brute_search(data, queries, 10, Euclidean());

In [8]:
idx, dist = search(knn_graph, queries, 10, 70);

In [9]:
recall(idx, true_idx)

0.9069999999999993

In [10]:
q_per_sec = length(queries)/(@belapsed search(knn_graph, queries, 10, 70))

115.55161439093644

In [11]:
@benchmark DescentGraph(data, 10)

BenchmarkTools.Trial: 
  memory estimate:  505.76 MiB
  allocs estimate:  10403588
  --------------
  minimum time:     994.717 ms (12.30% GC)
  median time:      1.038 s (14.30% GC)
  mean time:        1.040 s (15.08% GC)
  maximum time:     1.098 s (18.06% GC)
  --------------
  samples:          5
  evals/sample:     1

In [12]:
@benchmark search(knn_graph, queries, 10, 70)

BenchmarkTools.Trial: 
  memory estimate:  3.09 GiB
  allocs estimate:  28727345
  --------------
  minimum time:     4.448 s (8.51% GC)
  median time:      4.487 s (9.38% GC)
  mean time:        4.487 s (9.38% GC)
  maximum time:     4.527 s (10.23% GC)
  --------------
  samples:          2
  evals/sample:     1